In [1]:
import glob, os
import pandas as pd

In [2]:
transl_path = "/home/craig.car/repos/chiron/align_texts_project/data/translations_repositories.csv"

In [25]:
transl_df = pd.read_csv(transl_path)
transl_df = transl_df.rename(columns={"CTS-URN/s":"cts-urns_grp", "Original File Name/Location":"Original_File_Name", "Addiional File location/locations":"Additional_File_locations"})

In [26]:
transl_df.loc[transl_df["Editor; Translator"] == "Hickie, William James"]

,Author Name,WORK IDs,cts-urns_grp,# of translations,Edition Title,Editor; Translator,Publisher,Date,Original_File_Name,Additional_File_locations,Source Edition LInk,Translation Language,Worldcat Link,Work/s Split Up/Renamed in Branch,New File Name/s,Notes,TOC,Additional TOC
31,Aristophanes,0019.001 0019.002 0019.003 0019.004 0019.005 0...,urn:cts:greekLit:tlg0019.tlg001.ogl-eng1 urn:c...,11.0,"The Comedies of Aristophanes, Vol 1: Archarnia...","Hickie, William James",Bohn,1858,aristophanes_1_1858.xml,aristophanes_1907_1.xml,https://archive.org/details/cu31924087931675 (...,English,NaN,no,NaN,"Second file misname, it is volume 2 of this ed...",NaN,NaN
32,Aristophanes,NaN,NaN,5.0,"The Comedies of Aristophanes, Vol 2 (DUPLICATE)","Hickie, William James",Bohn,1853,aristophanes_2_1853.xml,NaN,https://archive.org/details/cu31924087931683,English,NaN,no,NaN,NaN,NaN,NaN
41,Aristophanes; Plautus,"0019.011, 119.10",urn:cts:greekLit:tlg0019.tlg011.ogl-eng3 urn:c...,2.0,"The Comedies of Aristophanes, Vol 6: The Plut...","Hickie, William James",Bell,1907,aristophanes_1907_2.xml,NaN,https://archive.org/details/ComediesOfAristoph...,English,NaN,no,NaN,Work contains a stealth translation of a work ...,NaN,NaN


In [27]:
transl_df.loc[0,"cts-urns_grp"]

'urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:cts:greekLit:tlg0085.tlg002.ogl-eng1 urn:cts:greekLit:tlg0085.tlg003.ogl-eng1 urn:cts:greekLit:tlg0085.tlg004.ogl-eng1 urn:cts:greekLit:tlg0085.tlg005.ogl-eng1 urn:cts:greekLit:tlg0085.tlg006.ogl-eng1 urn:cts:greekLit:tlg0085.tlg007.ogl-eng1'

In [28]:
transl_df.loc[0,"cts-urns_grp"].split()

['urn:cts:greekLit:tlg0085.tlg001.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg002.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg003.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg004.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg005.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg006.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg007.ogl-eng1']

In [29]:
transl_df["cts-urns_uniques"] = transl_df["cts-urns_grp"].str.split()

In [30]:
transl_df = transl_df.explode("cts-urns_uniques", ignore_index=True)

In [31]:
transl_df["ctsurns"] = transl_df["cts-urns_uniques"].str.rsplit(".",1).str[0]

/tmp/ipykernel_7626/1713395687.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  transl_df["ctsurns"] = transl_df["cts-urns_uniques"].str.rsplit(".",1).str[0]


In [32]:
transl_df["filename1"] = transl_df["Original_File_Name"].str.split(".").str[0]

In [33]:
transl_df["filename2"] = transl_df["Additional_File_locations"].str.split(".").str[0]

In [34]:
transl_df.head()

,Author Name,WORK IDs,cts-urns_grp,# of translations,Edition Title,Editor; Translator,Publisher,Date,Original_File_Name,Additional_File_locations,...,Worldcat Link,Work/s Split Up/Renamed in Branch,New File Name/s,Notes,TOC,Additional TOC,cts-urns_uniques,ctsurns,filename1,filename2
0,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1,urn:cts:greekLit:tlg0085.tlg001,aeschylus_1886,NaN
1,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg002.ogl-eng1,urn:cts:greekLit:tlg0085.tlg002,aeschylus_1886,NaN
2,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg003.ogl-eng1,urn:cts:greekLit:tlg0085.tlg003,aeschylus_1886,NaN
3,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg004.ogl-eng1,urn:cts:greekLit:tlg0085.tlg004,aeschylus_1886,NaN
4,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg005.ogl-eng1,urn:cts:greekLit:tlg0085.tlg005,aeschylus_1886,NaN


In [35]:
transl_df.isna().sum()

Author Name                            25
WORK IDs                               15
cts-urns_grp                           19
# of translations                       5
Edition Title                           0
Editor; Translator                      1
Publisher                               2
Date                                    4
Original_File_Name                      0
Additional_File_locations            1074
Source Edition LInk                    14
Translation Language                   22
Worldcat Link                        1292
Work/s Split Up/Renamed in Branch    1532
New File Name/s                      1575
Notes                                1239
TOC                                  1252
Additional TOC                       1547
cts-urns_uniques                       19
ctsurns                                19
filename1                               0
filename2                            1074
dtype: int64

In [48]:
transl_df.loc[transl_df["ctsurns"].isna()]

,Author Name,WORK IDs,cts-urns_grp,# of translations,Edition Title,Editor; Translator,Publisher,Date,Original_File_Name,Additional_File_locations,...,Worldcat Link,Work/s Split Up/Renamed in Branch,New File Name/s,Notes,TOC,Additional TOC,cts-urns_uniques,ctsurns,filename1,filename2
111,Aristophanes,NaN,NaN,5.0,"The Comedies of Aristophanes, Vol 2 (DUPLICATE)","Hickie, William James",Bohn,1853,aristophanes_2_1853.xml,NaN,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,aristophanes_2_1853,NaN
149,Aristophanes,NaN,NaN,NaN,"Aristophanes Lustspiele, I :Der Vogelstaat (18...","Minckwitz, Johannes, 1812-1885\n ...",Langenscheidt,1855,aristophanes_1855,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aristophanes_1855,NaN
160,Aristotle,NaN,NaN,2.0,Aristotle's Treatise on Rhetoric and The Poeti...,"Buckley, Theodore Alois, 1825-1856",Bohn,1857,aristotle_1857.xml,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aristotle_1857,NaN
161,Aristotle,NaN,NaN,2.0,Aristotle's Treatise on Rhetoric and The Poeti...,"Buckley, Theodore Alois, 1825-1856",Bohn,1906,aristotle_1906.xml,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aristotle_1906,NaN
164,Aristotle,NaN,NaN,2.0,The Politics and Economics of Aristotle (DUPLI...,"Walford, Edward",Bohn,1881,aristotle_1881.xml,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aristotle_1881,NaN
167,"Aristotle, Porphyry",NaN,NaN,6.0,"The Organon, or Logical Treatises of Aristotle...","Owen, Octavius Freire, 1816?-1873\n ...",Bohn,1853,aristotle_organon_1853_ddb1,aristotle_1853.xml,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aristotle_organon_1853_ddb1,aristotle_1853
175,Aristotle,NaN,NaN,0.0,Aristotle's History of Animals (DUPLICATE),"Cresswell, Richard",Bell,1883,aristotle_1883.xml,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aristotle_1883,NaN
181,Aristotle,NaN,NaN,1.0,Nikomachische Ethik (Duplicate),"Stahr, Adolf Wilhelm Theodor 1805-1876.",Krais & Hoffman,NaN,aristotle_1855-1906_1,NaN,...,NaN,NaN,NaN,Duplicate file with more errors,NaN,NaN,NaN,NaN,aristotle_1855-1906_1,NaN
309,Demosthenes,NaN,NaN,0.0,The Orations of Demosthenes on the Crown and t...,"Kennedy, Charles Rann",Bell,1865,demosthenes_1865,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,demosthenes_1865,NaN
459,Epictetus,NaN,NaN,4.0,The Discourses of Epictetus (DUPLICATE),"Long, George",Bell,1887,epictetus_1887,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,epictetus_1887,NaN


In [49]:
transl_df.fillna("", inplace=True)

In [50]:
transl_df["ctsurns"].duplicated().sum()

796

In [51]:
test = list(transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0006.tlg001", "filename1"])

In [52]:
test.extend(list(transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0006.tlg001", "filename2"]))

In [53]:
test

['euripides_1_1884',
 'euripides_1857_1',
 'euripides_1906_3',
 'euripides_1911',
 'euripides_2_1884',
 '',
 '',
 '']

In [54]:
transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0006.tlg001", "filename2"]

485    euripides_2_1884
493                    
494                    
500                    
Name: filename2, dtype: object

# Remove NAN from df?

In [17]:
import math

In [55]:
lookup_table = {}
keys = transl_df["ctsurns"].unique()
for ctsurn in keys:
    # lookup_table[ctsurn] = list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename"])
    vals = list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename1"])
    for other_name in list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename2"]):
        if other_name != "":
            vals.append(other_name)
    lookup_table[ctsurn] = vals

In [56]:
# spot check dict values
transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0085.tlg004", "filename1"]

3     aeschylus_1886
10    aeschylus_1888
17    aeschylus_1909
27    aeschylus_1872
37    aeschylus_1868
Name: filename1, dtype: object

In [59]:
lookup_table["urn:cts:greekLit:tlg0006.tlg001"]

['euripides_1_1884',
 'euripides_1857_1',
 'euripides_1906_3',
 'euripides_1911',
 'euripides_2_1884']

In [60]:
len("/scratch/craig.car/src_data/")

28

In [61]:
"/scratch/craig.car/src_data/urn:cts:greekLit:tlg0001.tlg001"[28:]

'urn:cts:greekLit:tlg0001.tlg001'

In [64]:
params = {
    'QUEUE':'short',
    'SWORKERS':2,
    'SMEM':4
}

labse_vec_cpu = 'srun --partition=short --nodes=1 --pty --mem=4GB --time=01:00:00 /bin/bash -c "source /home/craig.car/miniconda3/bin/activate; conda activate labse_vec_pipeline; {command};"'

src_dir = "/scratch/craig.car/src_data/"
fr_dir = "/scratch/craig.car/french_trans-dev/"

for src_emb in glob.iglob(src_dir+"*.emb"):
    ctsurn = os.path.splitext(os.path.basename(src_emb))[0]
    src_prefix = src_dir+ctsurn
    tgt_files = lookup_table[ctsurn]
    for tgt_name in tgt_files:
        tgt_prefix = fr_dir+tgt_name
        # print(tgt_name+".emb")
        # check if a fr translation exists
        if os.path.isfile(tgt_prefix+".emb") != False: 
            params['command'] = f'./vecalign.py --alignment_max_size 8 --src {src_prefix+".sents"} --tgt {tgt_prefix+".sents"} --src_embed {src_prefix+".overlaps"} {src_emb} --tgt_embed {tgt_prefix+".overlaps"} {tgt_prefix+".emb"}'
            print(labse_vec_cpu.format(**params))
            # subprocess.run(slurm_run.format(**params),shell=True)
    
            

srun --partition=short --nodes=1 --pty --mem=4GB --time=01:00:00 /bin/bash -c "source /home/craig.car/miniconda3/bin/activate; conda activate labse_vec_pipeline; ./vecalign.py --alignment_max_size 8 --src /scratch/craig.car/src_data/urn:cts:greekLit:tlg0003.tlg001.sents --tgt /scratch/craig.car/french_trans-dev/thucydides_1863.sents --src_embed /scratch/craig.car/src_data/urn:cts:greekLit:tlg0003.tlg001.overlaps /scratch/craig.car/src_data/urn:cts:greekLit:tlg0003.tlg001.emb --tgt_embed /scratch/craig.car/french_trans-dev/thucydides_1863.overlaps /scratch/craig.car/french_trans-dev/thucydides_1863.emb;"
srun --partition=short --nodes=1 --pty --mem=4GB --time=01:00:00 /bin/bash -c "source /home/craig.car/miniconda3/bin/activate; conda activate labse_vec_pipeline; ./vecalign.py --alignment_max_size 8 --src /scratch/craig.car/src_data/urn:cts:greekLit:tlg0003.tlg001.sents --tgt /scratch/craig.car/french_trans-dev/thucydides_1_1852.sents --src_embed /scratch/craig.car/src_data/urn:cts:gree